In [1]:
import pandas as pd
import random 
import itertools
import json
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, distance_matrix_calc, run_min_variance, run_clustering_model, test_for_silhouette_score

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)

joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level

In [3]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [4]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [5]:
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)

In [6]:
#Reassemble the results of the generation
# min_var_portfolios = dict()
# for i in range(1,1000):
#     with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
#         port = json.load(f)
#         min_var_portfolios.update(port)

In [7]:
# dtw_matrix = dtw_matrix_calc(joined_df)

# labels_kmeans, t, centroids_kmeans, centroids_kmeans = run_clustering_model(joined_df_weekly, n_clus=6, model_name='kmeans', return_mode='arithmetic')

# for i, centroid in enumerate(centroids_kmeans):
#     plt.plot(centroid.ravel(), label=f'Cluster {i}')
# plt.legend()
# plt.title("Cluster Centroids")


# labels_kshape = run_clustering_model(joined_df, n_clus=3, model_name='kshape')

# labels_ahc = run_clustering_model(joined_df, n_clus=3, model_name='ahc', linkage='single')

In [8]:
n_clusters_list = [4,5,6]


#kshape_inertia = test_for_silhouette_score(joined_df_weekly, n_clusters_list, method='kshape', return_mode='geometric')
#silhouette_df_ahc = test_for_silhouette_score(joined_df, n_clusters_list, method='ahc', linkage_list=['single', 'average', 'complete'])

for w_size in [3,5,7,10,14,21,30]:

#settings:
    return_mode = 'geometric'
    n_init = 3
    method = 'kmeans'
    window_size = w_size
    center = True

    moving_average_df = joined_df.rolling(window=window_size, center=center).mean() #try center False
    exponential_average_df = joined_df.ewm(span=window_size, adjust=False).mean()

    #CHANGE THE DF NAME!!!!!!!!!!!!!!!!!!

    df = 'exponential_average_df'

    silhouette_df_kmeans = test_for_silhouette_score(exponential_average_df, n_clusters_list, method=method, return_mode=return_mode, n_init=n_init)
    silhouette_df_kmeans['return_mode'] = return_mode
    silhouette_df_kmeans['n_init'] = n_init
    silhouette_df_kmeans['smoothing'] = df
    silhouette_df_kmeans['window_size/span'] = window_size

    silhouette_df_kmeans.to_csv(f'silhouette_dfs/456CLUSTERS{method}_{df}_window{window_size}_centered{center}_{return_mode}_{n_init}init.csv')

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = fun

In [9]:
# silhouette_df_kmeans['return_mode'] = return_mode
# silhouette_df_kmeans['n_init'] = n_init
# silhouette_df_kmeans['smoothing'] = df
# silhouette_df_kmeans['window_size/span'] = window_size

In [10]:
silhouette_df_kmeans

,clusters,silhouette_score,inertia,method,return_mode,n_init,smoothing,window_size/span
0,4,0.093438,68.786286,kmeans,geometric,3,exponential_average_df,30
1,5,0.077175,68.005017,kmeans,geometric,3,exponential_average_df,30
2,6,0.061179,63.851685,kmeans,geometric,3,exponential_average_df,30


In [11]:
# silhouette_df_kmeans.to_csv(f'silhouette_dfs/6CLUSTERS{method}_{df}_window{window_size}_centered{center}_{return_mode}_{n_init}init.csv')

In [12]:
# n_init_list = [3,5,10]
# dfs = dict()
# for n_init in n_init_list:
#     silhouette_df_kmeans = test_for_silhouette_score(joined_df_weekly, n_clusters_list, method='kmeans', return_mode='arithmetic', n_init_kmeans=n_init)
#     dfs[n_init] = silhouette_df_kmeans

# for df in dfs:
#     print(dfs[df])

In [13]:
# plt.figure(figsize=(8, 5))

# for linkage_type in silhouette_df_ahc['linkage'].unique():
#     subset = silhouette_df_ahc[silhouette_df_ahc['linkage'] == linkage_type]
#     plt.plot(subset['clusters'], subset['silhouette_score'],
#              marker='o',
#              label=f'Linkage: {linkage_type}')  # Use linkage as the label

# plt.xlabel("Number of clusters (k)")
# plt.ylabel("Silhouette Score")
# plt.title("Silhouette Scores by Linkage Method")
# plt.grid(True)
# plt.legend()
# plt.show()